<a href="https://colab.research.google.com/github/poojan14/AIMS/blob/main/BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [4]:
import pandas as pd
import numpy as np

download = drive.CreateFile({'id':'1QcgMQqHyY-Bx9XOd0Im3QIDaorf2n3YW'})
download.GetContentFile('feature_file(1).csv')
df1=pd.read_csv('feature_file(1).csv')

In [5]:
download = drive.CreateFile({'id':'1CQi9p06TptdOUgWm26KzJPG2toxSEhmd'})
download.GetContentFile('feature_file(2).csv')
df2=pd.read_csv('feature_file(2).csv')


In [6]:
download = drive.CreateFile({'id':'1C573UMtJt6vI8BzbTMQvWEq0rVO7Pw91'})
download.GetContentFile('feature_file(3).csv')
df3=pd.read_csv('feature_file(3).csv')


In [7]:
download = drive.CreateFile({'id':'10krvWfQNieGmCkNG6zahHSof-T7mqlVM'})
download.GetContentFile('feature_file(4).csv')
df4=pd.read_csv('feature_file(4).csv')


In [8]:
download = drive.CreateFile({'id':'10krvWfQNieGmCkNG6zahHSof-T7mqlVM'})
download.GetContentFile('feature_file(5).csv')
df5=pd.read_csv('feature_file(5).csv')


In [9]:
download = drive.CreateFile({'id':'1fpFBaiNbqydJUfhKdoKZZLqaTxRmJrOB'})
download.GetContentFile('feature_file(6).csv')
df6=pd.read_csv('feature_file(6).csv')


In [10]:
download = drive.CreateFile({'id':'15zRIpEb0006X6nup3r0gd-s-c86nrrqc'})
download.GetContentFile('feature_file(7).csv')
df7=pd.read_csv('feature_file(7).csv')


In [11]:
download = drive.CreateFile({'id':'117wn2jJg5EQWy53r4rhbrYAb9OBasMLj'})
download.GetContentFile('feature_file(8).csv')
df8=pd.read_csv('feature_file(8).csv')


In [12]:
download = drive.CreateFile({'id':'1B-dxpiLeMBJlY-K7zsE9Jtlsfdww73-j'})
download.GetContentFile('feature_file(9).csv')
df9=pd.read_csv('feature_file(9).csv')


In [13]:
download = drive.CreateFile({'id':'1K30DAz2rBA3070Xx1KbVTuV2LJ2vmK6n'})
download.GetContentFile('feature_file(10).csv')
df10=pd.read_csv('feature_file(10).csv')


In [14]:
df1=df1[df1.columns[1:6]] 
df2=df2[df2.columns[1:6]] 
df3=df3[df3.columns[1:6]] 
df4=df4[df4.columns[1:6]] 
df5=df5[df5.columns[1:6]] 
df6=df6[df6.columns[1:6]] 
df7=df7[df7.columns[1:6]] 
df8=df8[df8.columns[1:6]] 
df9=df9[df9.columns[1:6]] 
df10=df10[df10.columns[1:6]] 


In [15]:
df1=df1.dropna()
df2=df2.dropna()
df3=df3.dropna()
df4=df4.dropna()
df5=df5.dropna()
df6=df6.dropna()
df7=df7.dropna()
df8=df8.dropna()
df9=df9.dropna()
df10=df10.dropna()
df1=df1.dropna()


In [16]:
frames=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
df_tot=pd.concat(frames)

In [17]:
df_tot.head()

,BreathingDepth,BreathingRate,Cadence,HeartRate,Intensity
4,6.0,18.750000,0.0,61.000000,0.066881
6,3.0,22.058823,0.0,60.110094,0.058578
11,4.0,18.750000,0.0,58.153725,0.066881
14,4.0,22.058823,0.0,58.761372,0.087361
16,2.0,22.058823,0.0,59.609966,0.087361


In [18]:
df_tot.shape

(3057942, 5)

In [19]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf


In [20]:

column_indices = {name: i for i, name in enumerate(df_tot.columns)}

n = len(df_tot)
train_df = df_tot[0:int(n*0.7)]
val_df = df_tot[int(n*0.7):int(n*0.9)]
test_df = df_tot[int(n*0.9):]

num_features = df_tot.shape[1]

In [21]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [22]:
train_df.head(5)

,BreathingDepth,BreathingRate,Cadence,HeartRate,Intensity
4,-0.132821,-0.894633,-0.378789,-1.415598,-0.111473
6,-0.682219,-0.254040,-0.378789,-1.474966,-0.168167
11,-0.499086,-0.894633,-0.378789,-1.605480,-0.111473
14,-0.499086,-0.254040,-0.378789,-1.564943,0.028368
16,-0.865352,-0.254040,-0.378789,-1.508331,0.028368


In [23]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [24]:
w1 = WindowGenerator(input_width=288, label_width=288, shift=1,
                     label_columns=['BreathingRate'])
w2 = WindowGenerator(input_width=36, label_width=1, shift=1,
                     label_columns=['BreathingRate'])

In [25]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [26]:
# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)


In [27]:
w2.example = example_inputs, example_labels


In [28]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=64,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset


In [29]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example


In [30]:
# Each element is an (inputs, label) pair
w2.train.element_spec
for example_inputs, example_labels in w2.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')


Inputs shape (batch, time, features): (64, 36, 5)
Labels shape (batch, time, features): (64, 1, 1)


In [31]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you 
    # don't set the `label_columns` argument.
    input_width=36, label_width=1, shift=1)

wide_window = WindowGenerator(
    input_width=288, label_width=288, shift=1)

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (64, 288, 5)
Labels shape (batch, time, features): (64, 288, 5)


In [32]:
val_performance = {}
performance = {}


In [33]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history


In [35]:
%%time
wide_window = WindowGenerator(
    input_width=288, label_width=288, shift=1)

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]

    # Shape => [batch, time, features]
    tf.keras.layers.LSTM(64,return_sequences=True),
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()


9552/9552 [==============================] - 456s 48ms/step - loss: 0.4725 - mean_absolute_error: 0.3137

CPU times: user 12h 49min 47s, sys: 35min 28s, total: 13h 25min 16s
Wall time: 7h 21min 9s


In [33]:

class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each timestep is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta

In [34]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small
        # So initialize the output layer with zeros
        kernel_initializer=tf.initializers.zeros)
]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val)
performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0)
print()


9552/9552 [==============================] - 565s 59ms/step - loss: 0.4759 - mean_absolute_error: 0.3153

CPU times: user 8h 26min 21s, sys: 23min 9s, total: 8h 49min 31s
Wall time: 4h 52min 3s


In [34]:
import numpy as np
x = np.arange(len(performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = lstm_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_performance.values()]
test_mae = [v[metric_index] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
plt.ylabel('MAE (average over all outputs)')
_ = plt.legend()


NameError: ignored

In [36]:
for name, value in performance.items():
  print(f'{name:15s}: {value[1]:0.4f}')

Residual LSTM  : 0.3289


In [37]:
OUT_STEPS=24
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)


In [ ]:
feedback_model = FeedBack(units=144, out_steps=OUT_STEPS)


In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup


In [ ]:
prediction, state = feedback_model.warmup(wide_window.example[0])
prediction.shape

In [ ]:
%%time
prediction


In [ ]:
pred=prediction.numpy()


In [ ]:
pred=pd.DataFrame(pred)
print(pred)

In [ ]:
pred_mean = pred.mean()
pred_std = pred.std()

pred=(pred*pred_std) + pred_mean

In [ ]:
pred.head(5)

In [ ]:
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [ ]:
pred.to_csv('file1.csv') 
